In [1]:
# import packages needed for API access, data retrieval and data cleaning
import requests, json, os
import pandas as pd
import numpy as np
import time
import sqlite3
import warnings

from my_functions import *  # import all my custom functions needed to calc df transformations
from quickfs import QuickFS
from sqlalchemy import create_engine, inspect

In [2]:
# gain the ability to see all rows/columns if desired
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# ignore warnings
warnings.filterwarnings("ignore")  

In [3]:
# import api key and header for api access to quickfs.net
api_key = os.environ['QFS_KEY']
header = {'x-qfs-api-key': api_key}
client = QuickFS(api_key)

In [4]:
# # print all column names without quotes
# print(*final_df.columns, sep=', ')
# final_df.columns

In [5]:
# create the stocks sql database
engineStocks = create_engine("sqlite:///stocks.db", echo = False)
engineLeaders = create_engine("sqlite:///leaders.db", echo = False)
# connect to the stocks sql database
connStocks = sqlite3.connect('stocks.db')
connLeaders = sqlite3.connect('leaders.db')
# create the ability to perform logic on the database
s = connStocks.cursor()
l = connLeaders.cursor()
# create the stock table in the stocks database
s.execute('CREATE TABLE IF NOT EXISTS stocks_table '\
          '(Ticker TEXT, Name TEXT, Industry TEXT, Exchange TEXT, Market_Cap TEXT, Filing_Date TEXT, Q_End TEXT, '\
          'QNG_EPS INTEGER, QG_EPS INTEGER, QRev INTEGER, QRNG_EPS INTEGER, QRG_EPS INTEGER, QR_Rev INTEGER, '\
          'Q1_1 TEXT, Q1_2 TEXT, Q1_3 TEXT, Q2_1 TEXT, Q2_2 TEXT, Q2_3 TEXT, QS_1 TEXT, QS_2 TEXT, QS_3 TEXT, '\
          'R1_1 TEXT, R1_2 TEXT, R1_3 TEXT, R2_1 TEXT, R2_2 TEXT, R2_3 TEXT, RS_1 TEXT, RS_2 TEXT, RS_3 TEXT, '\
          'T1 TEXT, T2 TEXT, RT1 TEXT, RT2 TEXT, Q1 TEXT, Q2 TEXT, S TEXT, R1 TEXT, R2 TEXT, RS TEXT, '\
          'BO1 TEXT, BO2 TEXT, BOS TEXT, Total TEXT, '\
          'PRIMARY KEY (Ticker, Name, Industry, Exchange, Market_Cap, Filing_Date, Q_End, QNG_EPS, QG_EPS, QRev, '\
          'QRNG_EPS, QRG_EPS, QR_Rev, Q1_1, Q1_2, Q1_3, Q2_1, Q2_2, Q2_3, QS_1, QS_2, QS_3, R1_1, R1_2, R1_3, R2_1, R2_2, R2_3, '\
          'RS_1, RS_2, RS_3, T1, T2, RT1, RT2, Q1, Q2, S, R1, R2, RS, BO1, BO2, BOS, Total))')
l.execute('CREATE TABLE IF NOT EXISTS leaders_table '\
          '(Ticker TEXT, Name TEXT, Industry TEXT, Exchange TEXT, Market_Cap TEXT, Filing_Date TEXT, Q_End TEXT, '\
          'QNG_EPS INTEGER, QG_EPS INTEGER, QRev INTEGER, QRNG_EPS INTEGER, QRG_EPS INTEGER, QR_Rev INTEGER, '\
          'Q1_1 TEXT, Q1_2 TEXT, Q1_3 TEXT, Q2_1 TEXT, Q2_2 TEXT, Q2_3 TEXT, QS_1 TEXT, QS_2 TEXT, QS_3 TEXT, '\
          'R1_1 TEXT, R1_2 TEXT, R1_3 TEXT, R2_1 TEXT, R2_2 TEXT, R2_3 TEXT, RS_1 TEXT, RS_2 TEXT, RS_3 TEXT, '\
          'T1 TEXT, T2 TEXT, RT1 TEXT, RT2 TEXT, Q1 TEXT, Q2 TEXT, S TEXT, R1 TEXT, R2 TEXT, RS TEXT, '\
          'BO1 TEXT, BO2 TEXT, BOS TEXT, Total TEXT, '\
          'PRIMARY KEY (Ticker, Name, Industry, Exchange, Market_Cap, Filing_Date, Q_End, QNG_EPS, QG_EPS, QRev, '\
          'QRNG_EPS, QRG_EPS, QR_Rev, Q1_1, Q1_2, Q1_3, Q2_1, Q2_2, Q2_3, QS_1, QS_2, QS_3, R1_1, R1_2, R1_3, R2_1, R2_2, R2_3, '\
          'RS_1, RS_2, RS_3, T1, T2, RT1, RT2, Q1, Q2, S, R1, R2, RS, BO1, BO2, BOS, Total))')
# commit the stock table
connStocks.commit()
connLeaders.commit()
# check the stock table and make sure it was correctly added to the database
print(inspect(engineStocks).get_table_names())
print(inspect(engineLeaders).get_table_names())

['stocks_table']
['leaders_table']


In [6]:
# print total usage that day
client.get_usage()

{'quota': {'used': 0, 'remaining': 25000, 'resets': '2021-10-23T23:00:00Z'}}

In [7]:
## get a list of all stocks that QuickFS has on the NYSE
# nyse_list = client.get_supported_companies(country='US', exchange='NYSE')
# new_nyse_list = []
# for i in nyse_list:
#     new_nyse_list.append(i.split(':')[0])
# new_nyse_list = sorted(new_nyse_list)

## get a list of all stocks that QuickFS has on the NASDAQ
# nasdaq_list = client.get_supported_companies(country='US', exchange='NASDAQ')
# new_nasdaq_list = []
# for i in nasdaq_list:
#     new_nasdaq_list.append(i.split(':')[0])
# new_nasdaq_list = sorted(new_nasdaq_list)

In [ ]:
csv_df = pd.read_csv('StockList.csv')
stock_list = csv_df['Symbol'].tolist()[::-1]

In [ ]:
# create a count to keep track of stocks accessed
count = 0
# time how long it takes for the code to run
start_time = time.time()
# for loop to run through all stocks 
for i in stock_list:
    ticker = i
    country = 'US'

    # the api json request structure
    request_body = {
        "data" : {
            "name" : {
                "Name" : "QFS("+ticker+":"+country+",name)"
             },
            "industry" : {
                "Industry" : "QFS("+ticker+":"+country+",industry)"
             }, 
            "exchange" : {
                "Exchange": "QFS("+ticker+":"+country+",exchange)"
            },
            "mkt_cap" : {
                "Market_Cap": "QFS("+ticker+":"+country+",mkt_cap)"
            },
            "filing_date" : {
                "Filing_Date": "QFS("+ticker+":"+country+",original_filing_date,FQ:FQ)"
            },
            "q_end" : {
                "Q_End": "QFS("+ticker+":"+country+",period_end_date,FQ:FQ)"
            },
            "q_revenue" : {
                "Q_Revenue": "QFS("+ticker+":"+country+",revenue,FQ:FQ)"
            },
            "q_net_income" : {
                "Q_Net_Inc": "QFS("+ticker+":"+country+",net_income,FQ:FQ)"
            },
            "q_net_income_shareholders" : {
                "Q_Net_Inc_Avail_To_Common": "QFS("+ticker+":"+country+",net_income_available_to_shareholders,FQ:FQ)"
            },
            "q_shares_diluted" : {
                "Q_Dil_Share_Outs": "QFS("+ticker+":"+country+",shares_diluted,FQ:FQ)"
            },
            "q_ebitda" : {
                "Q_EBITDA": "QFS("+ticker+":"+country+",ebitda,FQ:FQ)"
            },
        }
    }

    r = requests.post("https://public-api.quickfs.net/v1/data/batch",json=request_body,headers=header)
    print(r.status_code, r.reason)
    print(ticker)
    if r.status_code == 200:
        # if status_code is normal, clean and insert data into data base
        my_data = r.json()
        # normalize the data (really 'flatten' the data) so it can be operated on
        temp_df = pd.json_normalize(my_data)
        # rename all columns
        new_columns = []
        [new_columns.append(i.split('.')[2]) for i in temp_df.columns]
        temp_df.columns = new_columns
        # reverse the rows of the data frame
        temp_df.insert(0, 'Ticker', ticker)
        temp_df = temp_df.apply(pd.Series.explode)
        temp_df = temp_df.iloc[::-1].reset_index(drop=True)
        # transform mrkt_cap column from num to str identifier
        temp_df['Market_Cap'] = temp_df['Market_Cap'].apply(mrk_cap)
        # create percent change columns for EPS G/NG, Rev and Profit Marg. G/NG
        temp_df['QNG_EPS'] = percent_change(division(temp_df['Q_EBITDA'], temp_df['Q_Dil_Share_Outs']))
        temp_df['QG_EPS'] = percent_change(division(temp_df['Q_Net_Inc_Avail_To_Common'], temp_df['Q_Dil_Share_Outs']))
        temp_df['QRev'] = percent_change(temp_df['Q_Revenue'])
        temp_df['QRNG_EPS'] = rolling_percent_change(division(temp_df['Q_EBITDA'], temp_df['Q_Dil_Share_Outs']))
        temp_df['QRG_EPS'] = rolling_percent_change(division(temp_df['Q_Net_Inc_Avail_To_Common'], temp_df['Q_Dil_Share_Outs']))
        temp_df['QR_Rev'] = rolling_percent_change(temp_df['Q_Revenue'])
        temp_df['QNG_PM'] = subtraction(division(temp_df['Q_EBITDA'], temp_df['Q_Revenue']))
        temp_df['QG_PM'] = subtraction(division(temp_df['Q_Net_Inc'], temp_df['Q_Revenue']))
        temp_df['QRNG_PM'] = rolling_subtraction(division(temp_df['Q_EBITDA'], temp_df['Q_Revenue']))
        temp_df['QRG_PM'] = rolling_subtraction(division(temp_df['Q_Net_Inc'], temp_df['Q_Revenue']))
        # symbology to visually represent percent buckets per column percent change column
        temp_df['Q1_1'] = eps_symbology1(temp_df['QNG_EPS'])
        temp_df['Q1_2'] = eps_symbology2(temp_df['QNG_EPS'])
        temp_df['Q1_3'] = eps_symbology3(temp_df['QNG_EPS'])
        temp_df['Q2_1'] = eps_symbology1(temp_df['QG_EPS'])
        temp_df['Q2_2'] = eps_symbology2(temp_df['QG_EPS'])
        temp_df['Q2_3'] = eps_symbology3(temp_df['QG_EPS'])
        temp_df['QS_1'] = rev_symbology1(temp_df['QRev'])
        temp_df['QS_2'] = rev_symbology2(temp_df['QRev'])
        temp_df['QS_3'] = rev_symbology3(temp_df['QRev'])
        temp_df['R1_1'] = eps_symbology1(temp_df['QRNG_EPS'])
        temp_df['R1_2'] = eps_symbology2(temp_df['QRNG_EPS'])
        temp_df['R1_3'] = eps_symbology3(temp_df['QRNG_EPS'])
        temp_df['R2_1'] = eps_symbology1(temp_df['QRG_EPS'])
        temp_df['R2_2'] = eps_symbology2(temp_df['QRG_EPS'])
        temp_df['R2_3'] = eps_symbology3(temp_df['QRG_EPS'])
        temp_df['RS_1'] = rev_symbology1(temp_df['QR_Rev'])
        temp_df['RS_2'] = rev_symbology2(temp_df['QR_Rev'])
        temp_df['RS_3'] = rev_symbology3(temp_df['QR_Rev'])
        # create all momentum indicator columns
        ## tenets
        temp_df['T1'] = tenet(temp_df['QNG_EPS'], temp_df['QRev'], temp_df['QNG_PM'])
        temp_df['T2'] = tenet(temp_df['QG_EPS'], temp_df['QRev'], temp_df['QG_PM'])
        temp_df['RT1'] = tenet(temp_df['QRNG_EPS'], temp_df['QR_Rev'], temp_df['QRNG_PM'])
        temp_df['RT2'] = tenet(temp_df['QRG_EPS'], temp_df['QR_Rev'], temp_df['QRG_PM'])
        ## momentum G/NG EPS, Rev
        temp_df['Q1'] = momentum(temp_df['QNG_EPS'])
        temp_df['Q2'] = momentum(temp_df['QG_EPS'])
        temp_df['S'] = momentum(temp_df['QRev'])
        temp_df['R1'] = momentum(temp_df['QRNG_EPS'])
        temp_df['R2'] = momentum(temp_df['QRG_EPS'])
        temp_df['RS'] = momentum(temp_df['QR_Rev'])
        ## breakouts G/NG EPS, Rev
        temp_df['BO1'] = breakout(temp_df['Q_EBITDA'], temp_df['Q_Dil_Share_Outs'])
        temp_df['BO2'] = breakout(temp_df['Q_Net_Inc_Avail_To_Common'], temp_df['Q_Dil_Share_Outs'])
        temp_df['BOS'] = sales_breakout(temp_df['Q_Revenue'])
        # drop columns no longer needed
        temp_df.drop(['Q_Revenue', 'Q_Net_Inc', 'Q_Net_Inc_Avail_To_Common', 'Q_Dil_Share_Outs', 'Q_EBITDA', \
                     'QNG_PM', 'QG_PM', 'QRNG_PM', 'QRG_PM'], axis=1, inplace=True)
        # replace infinite ('inf') values
        temp_df.replace(np.inf, '-', inplace=True)
        # create an empty scores dataframe
        score_df = pd.DataFrame()
        # scoring
        ## tenets
        score_df['T1'] = tenet_score(temp_df['T1'])
        score_df['T2'] = tenet_score(temp_df['T2'])
        score_df['RT1'] = tenet_score(temp_df['RT1'])
        score_df['RT2'] = tenet_score(temp_df['RT2'])
        ## financials
        score_df['NG'] = momentum_eps_score(temp_df['Q1'])
        score_df['NG1'] = eps_q1_score(temp_df['QNG_EPS'])
        score_df['NG2'] = eps_q2_score(temp_df['QNG_EPS'])
        score_df['NG3'] = eps_q3_score(temp_df['QNG_EPS'])

        score_df['G'] = momentum_eps_score(temp_df['Q2'])
        score_df['G1'] = eps_q1_score(temp_df['QG_EPS'])
        score_df['G2'] = eps_q2_score(temp_df['QG_EPS'])
        score_df['G3'] = eps_q3_score(temp_df['QG_EPS'])

        score_df['S'] = momentum_rev_score(temp_df['S'])
        score_df['S1'] = rev_q1_score(temp_df['QRev'])
        score_df['S2'] = rev_q2_score(temp_df['QRev'])
        score_df['S3'] = rev_q3_score(temp_df['QRev'])

        score_df['RNG'] = momentum_eps_score(temp_df['R1'])
        score_df['RNG1'] = eps_q1_score(temp_df['QRNG_EPS'])
        score_df['RNG2'] = eps_q2_score(temp_df['QRNG_EPS'])
        score_df['RNG3'] = eps_q3_score(temp_df['QRNG_EPS'])

        score_df['RG'] = momentum_eps_score(temp_df['R2'])
        score_df['RG1'] = eps_q1_score(temp_df['QRG_EPS'])
        score_df['RG2'] = eps_q2_score(temp_df['QRG_EPS'])
        score_df['RG3'] = eps_q3_score(temp_df['QRG_EPS'])

        score_df['RS'] = momentum_rev_score(temp_df['RS'])
        score_df['RS1'] = rev_q1_score(temp_df['QR_Rev'])
        score_df['RS2'] = rev_q2_score(temp_df['QR_Rev'])
        score_df['RS3'] = rev_q3_score(temp_df['QR_Rev'])
        ## breakouts
        score_df['BOG'] = eps_breakout(temp_df['BO1'])
        score_df['BONG'] = eps_breakout(temp_df['BO2'])
        score_df['BOS'] = rev_breakout(temp_df['BOS'])
        # produce the total score column
        score_df['Total'] = round(score_df.sum(axis=1))
        score_df['Total'].astype(int)
        # isolate the final total column
        score_df = score_df[['Total']]
        # concat temp_df and score_df to create the final df
        final_df = pd.concat([temp_df, score_df], axis=1)
        # remove the last four rows which have no momentum data of which we can use
        final_df.drop(final_df.tail(4).index, inplace=True)
    else: continue
    # add 1 to total count and print count
    count += 1
    # replace database if first pull, otherwise append data to its respective database
    if count == 1:
        final_df.to_sql('stocks_table', connStocks, if_exists='replace', index=False)
    else:
        final_df.to_sql('stocks_table', connStocks, if_exists='append', index=False)
    if count == 1:
        final_df.head(1).to_sql('leaders_table', connLeaders, if_exists='replace', index=False)
    else:
        final_df.head(1).to_sql('leaders_table', connLeaders, if_exists='append', index=False)
    print(final_df.shape)
    print(count)
    
print("This script took", round((time.time() - start_time)/60, 2), "minutes to run.")

In [ ]:
leaders = pd.read_sql_query('''SELECT * FROM "leaders_table"''', connLeaders)
leaders.head()

In [ ]:
leaders.shape

In [ ]:
stocks = pd.read_sql_query('''SELECT * FROM "stocks_table"''', connStocks)
stocks.head()

In [15]:
stocks.shape

(111092, 45)